In [8]:
import numpy as np
from astropy.io import fits
import glob
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
from scipy import signal
import re

In [9]:
def split_images(image_files, batch_size):
    return [image_files[i:i + batch_size] for i in range(0, len(image_files), batch_size)]

In [10]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [3]:
def DN2reflectance(image_filelist):
    result = []
    data = []
    t = []
    scale_factor = [1.254, 1, 0.645, 1.514]
    rad = 3.42 * pow(10,-3)
    for i, image_file in enumerate(image_filelist):
        hdu = fits.open(image_file)[0]
        header = hdu.header
        data.append(hdu.data)
        t.append(header["EXP_0"])
    n = image_filelist[0].split("/registration")
    ff = image_filelist
    name = n[0]
    save_dir = f"{name}/reflectance"
    print(save_dir)
    path = Path(save_dir)
    path.mkdir(exist_ok=True)
    base = (data[1] / t[1]) * rad
    for i in range(4):
        result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])
        ff = image_filelist[i].split("_reg_")
        f = ff[0].split("registration")
        fits.writeto(f"{f[0]}reflectance{f[1]}.fits", result[i], overwrite=True)
    return result

In [4]:
def find_greater_than_100_and_smallest_difference(value, lst):
    # 各要素と指定された値との差を計算
    differences = [(element, abs(element - value)) for element in lst if element > value and (element - value) >= 100]

    if not differences:
        return None  # 条件を満たす要素がない場合はNoneを返すか、適切な処理を行う

    # 差が小さい順にソート
    sorted_differences = sorted(differences, key=lambda x: x[1])

    # 指定された値よりも100以上大きく、かつ一番小さい差を持つ要素を取得
    result_element = sorted_differences[0][0]

    return result_element

In [141]:
def find_coordinates_cor2(image1, image2):
    coordinates_list = []
    above_threshold_coordinates = []
    rows, cols = image1.shape
    visited = np.zeros((rows, cols), dtype=bool)
    histogram, bin_edges = np.histogram(image1.flatten(), bins = 600, range = (1000, 4000))
    maxima_positions = signal.argrelmax(histogram.flatten(), order=1)[0]
    minima_positions = signal.argrelmin(histogram.flatten(), order=1)[0]
    place = np.argmax(histogram[maxima_positions])
    value = ((maxima_positions[place] * 5) + 1000) 
    minima_val = (minima_positions * 5)+1000
    maxima_val = (maxima_positions * 5)+1000
    closest_value = min((x for x in maxima_val if value + 100 < x <= value + 400), key=lambda x: abs(x - value), default=value)
    thresh = closest_value * 1.05

    def explore_area(i, j, threshold):
        area_coordinates = []
        stack = [(i, j)]

        while stack:
            current_i, current_j = stack.pop()
            if not visited[current_i, current_j] and image1[current_i, current_j] >= threshold and image2[current_i, current_j] != 0:#before2300
                visited[current_i, current_j] = True
                area_coordinates.append((current_i, current_j))

                # 近傍の未訪問ピクセルをスタックに追加
                for ni in range(current_i - 1, current_i + 2):
                    for nj in range(current_j - 1, current_j + 2):
                        if 0 <= ni < rows and 0 <= nj < cols and not visited[ni, nj] and image1[ni, nj] >= threshold and image2[current_i, current_j] != 0:
                            stack.append((ni, nj))

        return area_coordinates

    for i in range(rows):
        for j in range(cols):
            if not visited[i, j] and image1[i, j] >= thresh:
                area_coordinates = explore_area(i, j, thresh)
                if len(area_coordinates) >= 50:
                    coordinates_list.append(area_coordinates)

    for i in range(rows):
        for j in range(cols):
            if image1[i, j] > thresh and image2[i, j] != 0:
                above_threshold_coordinates.append((i, j))


    return coordinates_list, above_threshold_coordinates

In [138]:
# 20051119
def find_coordinates_cor1(image1, image2):
    coordinates_list = []
    above_threshold_coordinates = []
    rows, cols = image1.shape
    visited = np.zeros((rows, cols), dtype=bool)
    histogram, bin_edges = np.histogram(image1.flatten(), bins = 600, range = (1000, 4000))
    maxima_positions = signal.argrelmax(histogram.flatten(), order=1)[0]
    minima_positions = signal.argrelmin(histogram.flatten(), order=1)[0]
    place = np.argmax(histogram[maxima_positions])
    value = ((maxima_positions[place] * 5) + 1000) 
    minima_val = (minima_positions * 5)+1000
    maxima_val = (maxima_positions * 5)+1000
    closest_value = min((x for x in maxima_val if value + 100 < x <= value + 400), key=lambda x: abs(x - value), default=value)
    thresh = closest_value * 0.96

    def explore_area(i, j, thresh):
        area_coordinates = []
        stack = [(i, j)]

        while stack:
            current_i, current_j = stack.pop()
            if not visited[current_i, current_j] and image1[current_i, current_j] >= thresh and image2[current_i, current_j] != 0:#before 2000
                visited[current_i, current_j] = True
                area_coordinates.append((current_i, current_j))

                # 近傍の未訪問ピクセルをスタックに追加
                for ni in range(current_i - 1, current_i + 2):
                    for nj in range(current_j - 1, current_j + 2):
                        if 0 <= ni < rows and 0 <= nj < cols and not visited[ni, nj] and image1[ni, nj] >= thresh and image2[current_i, current_j] != 0:
                            stack.append((ni, nj))

        return area_coordinates

    for i in range(rows):
        for j in range(cols):
            if not visited[i, j] and image1[i, j] >= thresh:
                area_coordinates = explore_area(i, j, thresh)
                if len(area_coordinates) >= 50:
                    coordinates_list.append(area_coordinates)

    for i in range(rows):
        for j in range(cols):
            if image1[i, j] > thresh and image2[i, j] != 0:
                above_threshold_coordinates.append((i, j))

    return coordinates_list, above_threshold_coordinates

In [12]:
def create_image_from_coordinates(coordinates_list, image_shape, original_image):
    # 画像をゼロで初期化
    result_image = np.zeros(image_shape, dtype=original_image.dtype)

    # coordinates_listの各エリアに元の画像の値を設定
    for area_coordinates in coordinates_list:
        for coord in area_coordinates:
            result_image[coord[0], coord[1]] = original_image[coord[0], coord[1]]

    return result_image

In [142]:
def calc(image_filelist, target_dir, num):
    save_dir = f"../output_preprocess/{target_dir}/reflectance/"
    images = []
    for i in range(4):
        hdu = fits.open(image_filelist[i])[0]
        header = hdu.header
        images.append(hdu.data)
    # 条件を満たす座標を探す
    if "cor2" in image_filelist[0]:
        coordinates_list, above_coordinates_list = find_coordinates_cor2(images[0], images[3])
    # else:
    #     coordinates_list, above_coordinates_list = find_coordinates_cor2(images[0], images[3])
        
        reflectances = DN2reflectance(image_filelist)
        data = []
        for coord in above_coordinates_list:
            values_at_coordinate = [reflectances[f][coord[0], coord[1]] for f in range(4)]
            data.append([coord[1] + 1, coord[0] + 1] + values_at_coordinate)
        df = pd.DataFrame(data, columns=['座標_x', '座標_y', '反射率_b', '反射率_v', '反射率_w', '反射率_p'])
        df_r = df.replace([np.inf, -np.inf], np.nan).dropna()
        df_r.to_csv(f'{save_dir}{num}_.csv', index=False)

        data_coor = [(j + 1, i + 1) for i in range(images[0].shape[0]) for j in range(images[0].shape[1])]
        datas = pd.DataFrame({
        '反射率_b': reflectances[0].reshape(-1),
        '反射率_v': reflectances[1].reshape(-1),
        '反射率_w': reflectances[2].reshape(-1),
        '反射率_p': reflectances[3].reshape(-1)
        })
        # データフレームを構築
        df = pd.DataFrame(data_coor, columns=['座標_x', '座標_y'])
        df = pd.concat([df, datas], axis = 1)
        df = df[df["反射率_b"] != 0]
        df = df[df["反射率_w"] != 0]
        df = df[df["反射率_p"] != 0]
        df = df[df["反射率_v"] != 0]
        # df = df[df['座標_x'] >= 50]
        df_r = df.replace([np.inf, -np.inf], np.nan).dropna()
        df_r.to_csv(f'{save_dir}{num}_allcoord_.csv', index=False)

        if coordinates_list:
        # データを保持するDataFrameを作成
            df = pd.DataFrame(columns=['エリア', '座標_x', '座標_y', '反射率_b', '反射率_v', '反射率_w', '反射率_p'])
            i = 0
            for area_coordinates in coordinates_list:
                ref_b_at_coordinates = []
                ref_v_at_coordinates = []
                ref_w_at_coordinates = []
                ref_p_at_coordinates = []
                for coord in area_coordinates:
                    values_at_coordinate = [reflectances[f][coord[0], coord[1]] for f in range(4)]
                    if all(value != 0 for value in values_at_coordinate):
                        # df = df.replace([np.inf, -np.inf], np.nan).dropna()
                        df.loc[len(df)+1] = [int(i+1), coord[1]+1, coord[0]+1,
                                    values_at_coordinate[0], values_at_coordinate[1],
                                    values_at_coordinate[2], values_at_coordinate[3]]
                        ref_b_at_coordinates.append(values_at_coordinate[0])
                        ref_v_at_coordinates.append(values_at_coordinate[1])
                        ref_w_at_coordinates.append(values_at_coordinate[2])
                        ref_p_at_coordinates.append(values_at_coordinate[3])

                
                df = df.replace([np.inf, -np.inf], np.nan).dropna()
                if(any(df["エリア"]==i + 1)):

                    # DataFrameをCSVファイルに書き込み
                    wave_length = [429, 553, 700, 963]
                    # ref_area = ref_area.replace([np.inf, -np.inf], np.nan).dropna()
                    a = [np.mean(ref_b_at_coordinates), np.mean(ref_v_at_coordinates), np.mean(ref_w_at_coordinates), np.mean(ref_p_at_coordinates)]

                    plt.plot(wave_length, a, marker=".", label = f"area_{i+1}")
                    plt.xlabel('Wavelength (nm)')
                    plt.xlim(300, 1200)
                    plt.ylabel('reflectance normalized at 550')
                    plt.legend(loc='upper right')
                    plt.savefig(f"{save_dir}{num}_area{i+1}_avg.pdf")

                    if(((i+1) % 10) == 0):
                        plt.clf()
                    i +=1
            df = df[df["反射率_b"] != 0]
            df = df[df["反射率_w"] != 0]
            df = df[df["反射率_p"] != 0]
            df = df[df["反射率_v"] != 0]
            df.to_csv(f'{save_dir}{num}_area{i}.csv', index=False)
            print("エリアごとに保存しました.")
        else:
            print("保存された座標が見つかりませんでした.")
        plt.clf()

In [143]:
base_directory = f"../output_preprocess/"
for directory in sorted(os.listdir(base_directory),key = natural_keys):
    print(directory)
    if "2005" in directory:
        image_dir = f"../output_preprocess/{directory}/registration/"
        image_files = sorted(glob.glob(os.path.join(image_dir, "*.fits")))

        batch_size = 4

        image_batches = split_images(image_files, batch_size)
        for i, images in enumerate(image_batches):
            calc(images, directory, i)

20050917
../output_preprocess/20050917/reflectance


/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: divide by zero encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])
/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: invalid value encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])


エリアごとに保存しました.
20050918
../output_preprocess/20050918/reflectance


/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: divide by zero encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])
/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: invalid value encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])


エリアごとに保存しました.
20050919
../output_preprocess/20050919/reflectance


...

/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: divide by zero encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])
/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: invalid value encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])


エリアごとに保存しました.
../output_preprocess/20051025/reflectance


/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: divide by zero encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])
/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: invalid value encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])


エリアごとに保存しました.
20051103
20051104
20051119
.DS_Store


<Figure size 640x480 with 0 Axes>

In [17]:
target_dir = "20050923"
image_dir = f"../output_preprocess/{target_dir}/registration/"
image_files = sorted(glob.glob(os.path.join(image_dir, "*.fits")))

batch_size = 4

image_batches = split_images(image_files, batch_size)
print(image_batches[0])
for i, file_list in enumerate(image_batches):
    calc(file_list, target_dir, i)


['../output_preprocess/20050923/registration/cor2_st_2400804147_b_reg_.fits', '../output_preprocess/20050923/registration/cor2_st_2400810032_v_reg_.fits', '../output_preprocess/20050923/registration/cor2_st_2400829139_w_reg_.fits', '../output_preprocess/20050923/registration/cor2_st_2400834959_p_reg_.fits']
../output_preprocess/20050923/reflectance


/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: divide by zero encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])
/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: invalid value encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])


エリアごとに保存しました.
../output_preprocess/20050923/reflectance


/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: divide by zero encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])
/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: invalid value encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])


エリアごとに保存しました.
../output_preprocess/20050923/reflectance
保存された座標が見つかりませんでした.


/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: divide by zero encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])
/var/folders/pn/4j96z_ys4717vknhfxzddr900000gp/T/ipykernel_46617/308363484.py:21: RuntimeWarning: invalid value encountered in divide
  result.append((((data[i] / t[i])) / base) * rad * scale_factor[i])


TypeError: '>=' not supported between instances of 'int' and 'NoneType'

<Figure size 640x480 with 0 Axes>

In [35]:
target_dir = "20051013"
image_dir = f"../output_preprocess/{target_dir}/registration/"
image_files = sorted(glob.glob(os.path.join(image_dir, "*.fits")))

batch_size = 4

image_batches = split_images(image_files, batch_size)
print(image_batches[3])
images= []
for i, file_list in enumerate(image_batches[2]):
    images.append(fits.getdata(file_list))
image1 = images[0]
image2 = images[3]
coordinates_list = []
above_threshold_coordinates = []
rows, cols = image1.shape
visited = np.zeros((rows, cols), dtype=bool)
histogram, bin_edges = np.histogram(image1.flatten(), bins = 600, range = (1000, 4000))
maxima_positions = signal.argrelmax(histogram.flatten(), order=1)[0]
minima_positions = signal.argrelmin(histogram.flatten(), order=1)[0]
print(((maxima_positions * 5) + 1000))
place = np.argmax(histogram[maxima_positions])
value = (maxima_positions[place] * 5) + 1000
# thresh = find_greater_than_100_and_smallest_difference(value, (minima_positions * 5)+1000)
thresh = min(filter(lambda x: x > value*1.1, (minima_positions * 5)+1000), key=lambda x: abs(value - x), default=None)
print(thresh)

def explore_area(i, j, threshold):
    area_coordinates = []
    stack = [(i, j)]

    while stack:
        current_i, current_j = stack.pop()
        if not visited[current_i, current_j] and image1[current_i, current_j] >= threshold and image2[current_i, current_j] != 0:#before2300
            visited[current_i, current_j] = True
            area_coordinates.append((current_i, current_j))

            # 近傍の未訪問ピクセルをスタックに追加
            for ni in range(current_i - 1, current_i + 2):
                for nj in range(current_j - 1, current_j + 2):
                    if 0 <= ni < rows and 0 <= nj < cols and not visited[ni, nj] and image1[ni, nj] >= threshold and image2[current_i, current_j] != 0:
                        stack.append((ni, nj))

    return area_coordinates

for i in range(rows):
    for j in range(cols):
        if not visited[i, j] and image1[i, j] >= thresh:
            area_coordinates = explore_area(i, j, thresh)
            if len(area_coordinates) >= 100:
                coordinates_list.append(area_coordinates)

for i in range(rows):
    for j in range(cols):
        if image1[i, j] > thresh and image2[i, j] != 0:
            above_threshold_coordinates.append((i, j))


['../output_preprocess/20051013/registration/cor1_st_2457157533_b_reg_.fits', '../output_preprocess/20051013/registration/cor1_st_2457161422_v_reg_.fits', '../output_preprocess/20051013/registration/cor1_st_2457165278_w_reg_.fits', '../output_preprocess/20051013/registration/cor1_st_2457169150_p_reg_.fits']
[1025 1045 1075 1100 1120 1145 1155]
1140


In [42]:
hdu = fits.open("../output_preprocess/20051019/registration/cor2_st_2473318931_b_.fits")[0]
header = hdu.header


In [44]:
print(header)

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                  253                                                  NAXIS2  =                  255                                                  EXTEND  =                    T                                                  END                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [85]:
def find_min_max_coordinates(coordinates):
    if not coordinates:
        return None, None

    min_x, min_y = float('inf'), float('inf')
    max_x, max_y = float('-inf'), float('-inf')

    for x, y in coordinates:
        min_x = min(min_x, x)
        min_y = min(min_y, y)
        max_x = max(max_x, x)
        max_y = max(max_y, y)

    return (min_x, min_y), (max_x, max_y)

# 仮の座標リスト
coordinates = [(1, 3), (2, 5), (0, 2), (4, 1), (3, 4)]

# 最小の座標と最大の座標を求める
min_coord, max_coord = find_min_max_coordinates(coordinates)

print("最小の座標:", min_coord)
print("最大の座標:", max_coord)

最小の座標: (0, 1)
最大の座標: (4, 5)


In [144]:
import pandas as pd

# データフレームの作成
df = pd.DataFrame({'列1': [1, 2, 3], '列2': ['A', 'B', 'C']})

# 新しい行データと行ラベルを追加
new_row_data = [4, 'D']
new_row_label = '行4'

# 新しい行を追加
df.loc[new_row_label] = new_row_data


In [145]:
df

,列1,列2
0,1,A
1,2,B
2,3,C
行4,4,D
